In [1]:
# pip install -qU langchain-groq

In [2]:
import getpass
import os

# os.environ["GROQ_API_KEY"] = getpass.getpass()
os.environ["GROQ_API_KEY"]= "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"

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.1-8b-instant")

In [4]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import shutil
from langchain.storage._lc_store import create_kv_docstore
from langchain.storage import LocalFileStore
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
import pickle
import torch
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from langchain_core.documents import Document
import json

USER_AGENT environment variable not set, consider setting it to identify your requests.
/Users/mehuljain/miniconda3/envs/capstone_2/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


## loading stored embedding vectors

In [5]:
model_name = "Alibaba-NLP/gte-large-en-v1.5"
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"trust_remote_code": True}
)

In [6]:
import psutil

def check_ram():
    # Get memory information
    memory = psutil.virtual_memory()
    
    # Convert to GB for easier reading
    total = memory.total / (1024.0 ** 3)
    available = memory.available / (1024.0 ** 3)
    used = memory.used / (1024.0 ** 3)
    
    print(f"Total RAM: {total:.2f} GB")
    print(f"Available RAM: {available:.2f} GB")
    print(f"Used RAM: {used:.2f} GB")
    print(f"RAM Usage: {memory.percent}%")

check_ram()

Total RAM: 16.00 GB
Available RAM: 2.47 GB
Used RAM: 5.60 GB
RAM Usage: 84.5%


In [7]:
import torch

def check_mps_memory():
    if torch.backends.mps.is_available():
        print("MPS device is available.")
        
        # Get the total memory allocated by the Metal driver
        total_memory = torch.mps.current_allocated_memory()
        
        # Convert bytes to megabytes for easier reading
        total_memory_mb = total_memory / (1024 * 1024)
        
        print(f"Total GPU memory allocated: {total_memory_mb:.2f} MB")
    else:
        print("MPS device is not available.")

check_mps_memory()

MPS device is available.
Total GPU memory allocated: 1664.17 MB


In [8]:
parent_doc_path = "/Users/mehuljain/Documents/course_related/Capstone/rag_ds5500/vector_db/parent_docs"
child_doc_path = "../vector_db/child_docs"

In [9]:
# This text splitter is used to create the parent documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
# This text splitter is used to create the child documents
# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
# The vectorstore to use to index the child chunks

In [10]:
# Load Chroma vectorstore
loaded_vectorstore = Chroma(
    collection_name="split_parents",
    embedding_function=embeddings,
    persist_directory= child_doc_path
)
 
# Load parent document store
loaded_file_store = LocalFileStore(parent_doc_path)

In [11]:
class CustomParentDocumentRetriever(ParentDocumentRetriever):
    def invoke(self, query, config=None, **kwargs):
        results = super().invoke(query, config=config, **kwargs)
        documents = []
        for result in results:
            try:
                if isinstance(result, bytes):
                    json_string = result.decode('utf-8')
                elif isinstance(result, str):
                    json_string = result
                else:
                    print(f"Unexpected result type: {type(result)}")
                    continue

                deserialized_result = json.loads(json_string)
                
                # Extract metadata and page_content from the deserialized result
                metadata = deserialized_result['kwargs']['metadata']
                page_content = deserialized_result['kwargs']['page_content']

                doc = Document(metadata=metadata, page_content=page_content)
                documents.append(doc)
            except Exception as e:
                print(f"Error processing result: {e}")
                print(f"Problematic result: {result}")

        return documents

In [12]:
# Use the custom retriever
loaded_retriever = CustomParentDocumentRetriever(
    vectorstore=loaded_vectorstore,
    docstore=loaded_file_store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)

In [13]:
len(list(loaded_file_store.yield_keys()))

13720

In [14]:
check_mps_memory()

MPS device is available.
Total GPU memory allocated: 1664.17 MB


In [15]:
loaded_vectorstore.similarity_search("how can i create a 2d array?")

[Document(metadata={'doc_id': '631be762-2f59-49e2-a8f1-03975d3aa39b', 'source': 'https://numpy.org/doc/stable/reference/arrays.ndarray.html', 'title': 'The N-dimensional array (ndarray) — NumPy v2.0 Manual'}, page_content='>>> y = x[:,1]\n>>> y\narray([2, 5], dtype=int32)\n>>> y[0] = 9 # this also changes the corresponding element in x\n>>> y\narray([9, 5], dtype=int32)\n>>> x\narray([[1, 9, 3],\n       [4, 5, 6]], dtype=int32)\n\n\nConstructing arrays#\n\nNew arrays can be constructed using the routines detailed inArray creation routines, and also by using the low-levelndarrayconstructor:\n\nndarray(shape[,\xa0dtype,\xa0buffer,\xa0offset,\xa0...])\n\nAn array object represents a multidimensional, homogeneous array of fixed-size items.'),
 Document(metadata={'doc_id': '9c040906-0485-4022-83fd-0be4b34b81ff', 'source': 'https://numpy.org/doc/stable/user/absolute_beginners.html', 'title': 'NumPy: the absolute basics for beginners — NumPy v2.0 Manual'}, page_content='How to create a basic 

In [16]:
rt=loaded_retriever.invoke("how can i create a 2d array?")
rt

[Document(metadata={'source': 'https://numpy.org/doc/stable/reference/arrays.ndarray.html', 'title': 'The N-dimensional array (ndarray) — NumPy v2.0 Manual'}, page_content="The N-dimensional array (ndarray)#\nndarray\nAnndarrayis a (usually fixed-size) multidimensional\ncontainer of items of the same type and size. The number of dimensions\nand items in an array is defined by itsshape,\nwhich is atupleofNnon-negative integers that specify the\nsizes of each dimension. The type of items in the array is specified by\na separatedata-type object (dtype), one of which\nis associated with each ndarray.\nshapetuple\nAs with other container objects in Python, the contents of anndarraycan be accessed and modified byindexing or\nslicingthe array (using, for example,Nintegers),\nand via the methods and attributes of thendarray.\n\nDifferentndarrayscan share the same data, so that\nchanges made in onendarraymay be visible in another. That\nis, an ndarray can be a“view”to another ndarray, and the d

In [17]:
rt[0].metadata['source']

'https://numpy.org/doc/stable/reference/arrays.ndarray.html'

In [18]:
for i in rt:
    print(i.metadata['source'])

https://numpy.org/doc/stable/reference/arrays.ndarray.html
https://numpy.org/doc/stable/user/absolute_beginners.html
https://numpy.org/doc/stable/reference/generated/numpy.ma.MaskedArray.count.html
https://numpy.org/doc/stable/reference/generated/numpy.ma.masked_array.count.html


In [19]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(loaded_retriever, question_answer_chain)

In [20]:
response = rag_chain.invoke({"input": "how can i create a 2d array? Give detailed instructions"})

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [21]:
print(f"Answer: {response['answer']}")

Answer: To create a 2D array in NumPy, you can use the `np.ones()`, `np.zeros()`, or `np.random()` functions, and pass a tuple describing the dimensions of the matrix as an argument.

Here are the detailed instructions:

1. Import the NumPy library: `import numpy as np`
2. Choose the function you want to use:
	* `np.ones()` to create an array filled with ones
	* `np.zeros()` to create an array filled with zeros
	* `np.random()` to create an array filled with random numbers
3. Pass a tuple describing the dimensions of the matrix as an argument. For example:
	* To create a 2x3 matrix, use `(2, 3)`
	* To create a 3x4 matrix, use `(3, 4)`
4. Call the function with the tuple as an argument. For example:
	* `np.ones((2, 3))` to create a 2x3 matrix filled with ones
	* `np.zeros((3, 4))` to create a 3x4 matrix filled with zeros
	* `np.random((3, 4))` to create a 3x4 matrix filled with random numbers

Here are some examples:

* `np.ones((2, 3))` returns `array([[1., 1., 1.], [1., 1., 1.]])`
* `

## Adding chat history

In [22]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [23]:
### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, loaded_retriever, contextualize_q_prompt
)

In [24]:
### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [25]:
### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [26]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]


"I don't know. The provided context does not mention task decomposition."

In [27]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?'), AIMessage(content="I don't know. The provided context does not mention task decomposition.")])}

In [28]:
conversational_rag_chain.invoke(
    {"input": "how can i create a 2d array?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'You can create a 2D array using the `np.array()` function or by using the `np.zeros()`, `np.ones()`, or `np.empty()` functions with a tuple specifying the shape of the array.'

In [29]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?'), AIMessage(content="I don't know. The provided context does not mention task decomposition."), HumanMessage(content='how can i create a 2d array?'), AIMessage(content='You can create a 2D array using the `np.array()` function or by using the `np.zeros()`, `np.ones()`, or `np.empty()` functions with a tuple specifying the shape of the array.')])}

In [30]:
conversational_rag_chain.invoke(
    {"input": "can you elaborate more on a = np.arange(4).reshape(2,2)?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'`np.arange(4)` creates an array `[0, 1, 2, 3]`. The `reshape(2,2)` method then changes the shape of this array to a 2x2 matrix. The elements of the original array are rearranged to fill the new shape. So, `a` becomes `[[0, 1], [2, 3]]`.'

In [31]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?'), AIMessage(content="I don't know. The provided context does not mention task decomposition."), HumanMessage(content='how can i create a 2d array?'), AIMessage(content='You can create a 2D array using the `np.array()` function or by using the `np.zeros()`, `np.ones()`, or `np.empty()` functions with a tuple specifying the shape of the array.'), HumanMessage(content='can you elaborate more on a = np.arange(4).reshape(2,2)?'), AIMessage(content='`np.arange(4)` creates an array `[0, 1, 2, 3]`. The `reshape(2,2)` method then changes the shape of this array to a 2x2 matrix. The elements of the original array are rearranged to fill the new shape. So, `a` becomes `[[0, 1], [2, 3]]`.')])}

In [32]:
conversational_rag_chain.invoke(
    {"input": "can you give some more examples and use cases?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Here are a few more examples of creating 2D arrays using `np.arange()` and `reshape()`:\n\n```python\nimport numpy as np\n\n# Create a 1D array of numbers from 0 to 8\na = np.arange(9)\nprint(a)  # [0, 1, 2, 3, 4, 5, 6, 7, 8]\n\n# Reshape it into a 3x3 array\na = a.reshape(3, 3)\nprint(a)  # [[0, 1, 2],\n          #  [3, 4, 5],\n          #  [6, 7, 8]]\n\n# Create a 1D array of numbers from 0 to 11\na = np.arange(12)\nprint(a)  # [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]\n\n# Reshape it into a 4x3 array\na = a.reshape(4, 3)\nprint(a)  # [[0, 1, 2],\n          #  [3, 4, 5],\n          #  [6, 7, 8],\n          #  [9, 10, 11]]\n```\n\nThis can be useful when working with data that has a natural 2D structure, such as matrices, grids, or images. The `reshape()` method allows you to easily convert between different 2D shapes, which can make your code more readable and efficient.'

In [45]:
store['abc123'].messages[-1].content

'Here are a few more examples of creating 2D arrays using `np.arange()` and `reshape()`:\n\n```python\nimport numpy as np\n\n# Create a 1D array of numbers from 0 to 8\na = np.arange(9)\nprint(a)  # [0, 1, 2, 3, 4, 5, 6, 7, 8]\n\n# Reshape it into a 3x3 array\na = a.reshape(3, 3)\nprint(a)  # [[0, 1, 2],\n          #  [3, 4, 5],\n          #  [6, 7, 8]]\n\n# Create a 1D array of numbers from 0 to 11\na = np.arange(12)\nprint(a)  # [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]\n\n# Reshape it into a 4x3 array\na = a.reshape(4, 3)\nprint(a)  # [[0, 1, 2],\n          #  [3, 4, 5],\n          #  [6, 7, 8],\n          #  [9, 10, 11]]\n```\n\nThis can be useful when working with data that has a natural 2D structure, such as matrices, grids, or images. The `reshape()` method allows you to easily convert between different 2D shapes, which can make your code more readable and efficient.'

In [33]:
print(conversational_rag_chain.invoke(
    {"input": "can you give some more examples and use cases?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"])

KeyboardInterrupt: 

In [ ]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?'), AIMessage(content="I don't know. The provided context does not mention Task Decomposition."), HumanMessage(content='how can i create a 2d array?'), AIMessage(content='You can create a 2D array in NumPy using the np.array() function with a list of lists. For example: `a = np.array([[1, 2, 3], [4, 5, 6]])`. Alternatively, you can use the np.arange() function and the reshape() method, as shown in the example: `a = np.arange(4).reshape(2,2)`.'), HumanMessage(content='can you elaborate more on a = np.arange(4).reshape(2,2)?'), AIMessage(content='`np.arange(4)` generates an array of numbers from 0 to 3, which is a 1D array: `[0, 1, 2, 3]`. The `reshape(2,2)` method then reshapes this array into a 2D array with 2 rows and 2 columns: `[[0, 1], [2, 3]]`.'), HumanMessage(content='can you give some more examples and use cases?'), AIMessage(content='Here are a few more examples:\n\n* `a = np.arange

In [ ]:
print(conversational_rag_chain.invoke(
    {"input": "who are you?"},
    config={"configurable": {"session_id": "xyz456"}},
)["answer"])

I'm an assistant for question-answering tasks. I was trained on a large corpus of text and can provide concise answers to a wide range of questions. I don't have personal opinions or emotions, and I don't know everything, but I'll do my best to provide a helpful response to your question.


In [ ]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?'), AIMessage(content="I don't know. The provided context does not mention Task Decomposition."), HumanMessage(content='how can i create a 2d array?'), AIMessage(content='You can create a 2D array in NumPy using the np.array() function with a list of lists. For example: `a = np.array([[1, 2, 3], [4, 5, 6]])`. Alternatively, you can use the np.arange() function and the reshape() method, as shown in the example: `a = np.arange(4).reshape(2,2)`.'), HumanMessage(content='can you elaborate more on a = np.arange(4).reshape(2,2)?'), AIMessage(content='`np.arange(4)` generates an array of numbers from 0 to 3, which is a 1D array: `[0, 1, 2, 3]`. The `reshape(2,2)` method then reshapes this array into a 2D array with 2 rows and 2 columns: `[[0, 1], [2, 3]]`.'), HumanMessage(content='can you give some more examples and use cases?'), AIMessage(content='Here are a few more examples:\n\n* `a = np.arange